# Combined Analysis: Style Bias × Social Bias

This notebook compares results from Test 2 (style bias) and Test 3 (social bias) to answer:
- How does an **overstated resume** compare to different **demographic groups**?
- Can style compensate for demographic bias? (e.g., Can an understated Caucasian beat an overstated African American?)
- Which combination gets the highest/lowest predicted seniority?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Seniority ranking for comparison
seniority_rank = {'junior': 0, 'mid': 1, 'senior': 2}

## Load Results from Both Tests

In [ ]:
# Load Test 2 results (style bias)
# Try finetuned models first, then LLM if available
try:
    test2_ft = pd.read_csv("../Test 2/test2_finetuned_predictions.csv")
    print(f"Loaded Test 2 finetuned: {len(test2_ft)} predictions")
except:
    test2_ft = None
    print("Test 2 finetuned predictions not found")

try:
    test2_llm = pd.read_csv("../Test 2/test2_llm_predictions.csv")
    print(f"Loaded Test 2 LLM: {len(test2_llm)} predictions")
except:
    test2_llm = None
    print("Test 2 LLM predictions not found")

In [ ]:
# Load Test 3 results (social bias)
try:
    test3_ft = pd.read_csv("../Test 3/test3_finetuned_predictions.csv")
    print(f"Loaded Test 3 finetuned: {len(test3_ft)} predictions")
except:
    test3_ft = None
    print("Test 3 finetuned predictions not found")

try:
    test3_llm = pd.read_csv("../Test 3/test3_llm_predictions.csv")
    print(f"Loaded Test 3 LLM: {len(test3_llm)} predictions")
except:
    test3_llm = None
    print("Test 3 LLM predictions not found")

---
## Analysis 1: Style vs Demographics - Who Gets Senior?

**What it is:** Compare the rate of "Senior" predictions between:
- Different styles (overstated, neutral, understated) from Test 2
- Different demographics from Test 3

**Why we do it:** To see if writing style matters more than demographics (or vice versa).

**How to read:** Higher % = model predicts Senior more often for that group

In [ ]:
def calc_senior_rate(df):
    """Calculate % of predictions that are 'senior'"""
    if df is None or len(df) == 0:
        return None
    valid = df[~df['prediction'].isin(['error', 'unknown'])]
    return (valid['prediction'] == 'senior').mean() * 100

def calc_avg_rank(df):
    """Calculate average predicted rank (0=junior, 1=mid, 2=senior)"""
    if df is None or len(df) == 0:
        return None
    valid = df[~df['prediction'].isin(['error', 'unknown'])].copy()
    valid['pred_rank'] = valid['prediction'].map(seniority_rank)
    return valid['pred_rank'].mean()

In [ ]:
print("=" * 70)
print("STYLE vs DEMOGRAPHICS: Senior Prediction Rate")
print("=" * 70)

# Get unique models from available data
models = set()
if test2_ft is not None:
    models.update(test2_ft['model'].unique())
if test3_ft is not None:
    models.update(test3_ft['model'].unique())
if test2_llm is not None:
    models.update(test2_llm['model'].unique())
if test3_llm is not None:
    models.update(test3_llm['model'].unique())

for model in sorted(models):
    print(f"\n{model.upper()}")
    print("-" * 50)
    
    # Style rates from Test 2
    print("\nBy STYLE (Test 2):")
    for test2 in [test2_ft, test2_llm]:
        if test2 is not None and model in test2['model'].values:
            for style in ['overstated', 'neutral', 'understated']:
                style_df = test2[(test2['model'] == model) & (test2['style'] == style)]
                sr = calc_senior_rate(style_df)
                ar = calc_avg_rank(style_df)
                if sr is not None:
                    print(f"  {style:<15}: {sr:5.1f}% senior (avg rank: {ar:.2f})")
            break
    
    # Demographic rates from Test 3
    print("\nBy DEMOGRAPHIC (Test 3):")
    for test3 in [test3_ft, test3_llm]:
        if test3 is not None and model in test3['model'].values:
            for demo in ['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female']:
                demo_df = test3[(test3['model'] == model) & (test3['demographic'] == demo)]
                sr = calc_senior_rate(demo_df)
                ar = calc_avg_rank(demo_df)
                if sr is not None:
                    print(f"  {demo:<25}: {sr:5.1f}% senior (avg rank: {ar:.2f})")
            break

---
## Analysis 2: Can Style Beat Demographics?

**What it is:** Compare specific style+demographic combinations:
- Overstated (any demographic) vs Understated Caucasian
- Understated (any demographic) vs Overstated African American

**Why we do it:** To see if how you write matters more than who you are.

**How to read:** If overstated gets higher rank than neutral demographic = style > demographics

In [ ]:
print("=" * 70)
print("CAN STYLE BEAT DEMOGRAPHICS?")
print("=" * 70)

for model in sorted(models):
    print(f"\n{model.upper()}")
    print("-" * 60)
    
    style_ranks = {}
    demo_ranks = {}
    
    # Get style ranks
    for test2 in [test2_ft, test2_llm]:
        if test2 is not None and model in test2['model'].values:
            for style in ['overstated', 'neutral', 'understated']:
                style_df = test2[(test2['model'] == model) & (test2['style'] == style)]
                style_ranks[style] = calc_avg_rank(style_df)
            break
    
    # Get demographic ranks
    for test3 in [test3_ft, test3_llm]:
        if test3 is not None and model in test3['model'].values:
            for demo in ['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female']:
                demo_df = test3[(test3['model'] == model) & (test3['demographic'] == demo)]
                demo_ranks[demo] = calc_avg_rank(demo_df)
            break
    
    if style_ranks and demo_ranks:
        # Compare: Overstated vs best demographic
        best_demo = max(demo_ranks, key=demo_ranks.get)
        worst_demo = min(demo_ranks, key=demo_ranks.get)
        
        print(f"\nRanking comparison (0=junior, 1=mid, 2=senior):")
        print(f"  Overstated resume:       {style_ranks.get('overstated', 'N/A'):.2f}")
        print(f"  Best demographic ({best_demo[:10]}): {demo_ranks[best_demo]:.2f}")
        print(f"  Neutral resume:          {style_ranks.get('neutral', 'N/A'):.2f}")
        print(f"  Understated resume:      {style_ranks.get('understated', 'N/A'):.2f}")
        print(f"  Worst demographic ({worst_demo[:10]}): {demo_ranks[worst_demo]:.2f}")
        
        # Key comparisons
        overstated = style_ranks.get('overstated', 0)
        understated = style_ranks.get('understated', 0)
        
        print(f"\n  KEY FINDINGS:")
        if overstated > demo_ranks[best_demo]:
            print(f"  ✓ Overstated beats best demographic (+{overstated - demo_ranks[best_demo]:.2f})")
        else:
            print(f"  ✗ Best demographic beats overstated (+{demo_ranks[best_demo] - overstated:.2f})")
        
        if understated < demo_ranks[worst_demo]:
            print(f"  ✓ Understated worse than worst demographic ({understated:.2f} < {demo_ranks[worst_demo]:.2f})")
        else:
            print(f"  ✗ Worst demographic worse than understated ({demo_ranks[worst_demo]:.2f} < {understated:.2f})")
        
        style_gap = overstated - understated
        demo_gap = demo_ranks[best_demo] - demo_ranks[worst_demo]
        print(f"\n  Style impact (overstated - understated): {style_gap:+.2f}")
        print(f"  Demographic impact (best - worst):       {demo_gap:+.2f}")
        
        if style_gap > demo_gap:
            print(f"\n  → STYLE has MORE impact than DEMOGRAPHICS")
        else:
            print(f"\n  → DEMOGRAPHICS has MORE impact than STYLE")

---
## Analysis 3: Combined Ranking (All Combinations)

**What it is:** Rank all style and demographic groups together.

**Why we do it:** To see the full picture of what gets the highest predictions.

In [ ]:
print("=" * 70)
print("COMBINED RANKING: ALL GROUPS")
print("=" * 70)

for model in sorted(models):
    print(f"\n{model.upper()}")
    print("-" * 50)
    
    all_groups = []
    
    # Add style groups
    for test2 in [test2_ft, test2_llm]:
        if test2 is not None and model in test2['model'].values:
            for style in ['overstated', 'neutral', 'understated']:
                style_df = test2[(test2['model'] == model) & (test2['style'] == style)]
                rank = calc_avg_rank(style_df)
                if rank is not None:
                    all_groups.append({'group': f"[STYLE] {style}", 'avg_rank': rank, 'type': 'style'})
            break
    
    # Add demographic groups
    for test3 in [test3_ft, test3_llm]:
        if test3 is not None and model in test3['model'].values:
            for demo in ['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female']:
                demo_df = test3[(test3['model'] == model) & (test3['demographic'] == demo)]
                rank = calc_avg_rank(demo_df)
                if rank is not None:
                    all_groups.append({'group': f"[DEMO] {demo}", 'avg_rank': rank, 'type': 'demo'})
            break
    
    # Sort and display
    all_groups.sort(key=lambda x: x['avg_rank'], reverse=True)
    
    print(f"{'Rank':<6} {'Group':<35} {'Avg Predicted Rank'}")
    print("-" * 60)
    for i, g in enumerate(all_groups, 1):
        print(f"{i:<6} {g['group']:<35} {g['avg_rank']:.3f}")

---
## Visualization: Style vs Demographics Impact

In [ ]:
# Get first available model for visualization
plot_model = list(models)[0] if models else None

if plot_model:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    groups = []
    ranks = []
    colors = []
    
    # Style data
    for test2 in [test2_ft, test2_llm]:
        if test2 is not None and plot_model in test2['model'].values:
            for style in ['overstated', 'neutral', 'understated']:
                style_df = test2[(test2['model'] == plot_model) & (test2['style'] == style)]
                rank = calc_avg_rank(style_df)
                if rank is not None:
                    groups.append(f"[S] {style[:5]}")
                    ranks.append(rank)
                    colors.append('#3498db')  # Blue for style
            break
    
    # Demographic data
    demo_labels = ['Cauc.M', 'Cauc.F', 'AA.M', 'AA.F']
    for test3 in [test3_ft, test3_llm]:
        if test3 is not None and plot_model in test3['model'].values:
            for demo, label in zip(['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female'], demo_labels):
                demo_df = test3[(test3['model'] == plot_model) & (test3['demographic'] == demo)]
                rank = calc_avg_rank(demo_df)
                if rank is not None:
                    groups.append(f"[D] {label}")
                    ranks.append(rank)
                    colors.append('#e74c3c')  # Red for demographics
            break
    
    if groups:
        x = np.arange(len(groups))
        bars = ax.bar(x, ranks, color=colors)
        
        ax.set_ylabel('Avg Predicted Rank (0=Junior, 1=Mid, 2=Senior)')
        ax.set_title(f'Style [Blue] vs Demographics [Red] - {plot_model.upper()}')
        ax.set_xticks(x)
        ax.set_xticklabels(groups, rotation=45, ha='right')
        ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Mid level')
        
        plt.tight_layout()
        plt.show()
else:
    print("No prediction data available for visualization")

---
## Summary

**Interpretation Guide:**

1. **If Style impact > Demographic impact:** How you write your resume matters more than your name/perceived identity. This suggests models focus on content/tone rather than demographic cues.

2. **If Demographic impact > Style impact:** Your perceived identity (based on name) has more influence than how you present yourself. This indicates social bias in the model.

3. **If both are significant:** Both factors play a role, and the worst case would be an understated resume from a disadvantaged demographic group.

4. **Ideal case:** Neither should significantly affect predictions - only actual qualifications matter.

In [ ]:
print("=" * 70)
print("FINAL SUMMARY: STYLE vs SOCIAL BIAS")
print("=" * 70)

for model in sorted(models):
    print(f"\n{model.upper()}")
    print("-" * 50)
    
    # Calculate impacts
    style_max, style_min = None, None
    demo_max, demo_min = None, None
    
    for test2 in [test2_ft, test2_llm]:
        if test2 is not None and model in test2['model'].values:
            ranks = []
            for style in ['overstated', 'neutral', 'understated']:
                r = calc_avg_rank(test2[(test2['model'] == model) & (test2['style'] == style)])
                if r is not None:
                    ranks.append(r)
            if ranks:
                style_max, style_min = max(ranks), min(ranks)
            break
    
    for test3 in [test3_ft, test3_llm]:
        if test3 is not None and model in test3['model'].values:
            ranks = []
            for demo in ['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female']:
                r = calc_avg_rank(test3[(test3['model'] == model) & (test3['demographic'] == demo)])
                if r is not None:
                    ranks.append(r)
            if ranks:
                demo_max, demo_min = max(ranks), min(ranks)
            break
    
    if style_max is not None and demo_max is not None:
        style_impact = style_max - style_min
        demo_impact = demo_max - demo_min
        
        print(f"Style impact (max - min):       {style_impact:.3f}")
        print(f"Demographic impact (max - min): {demo_impact:.3f}")
        
        if style_impact > demo_impact * 1.5:
            print(f"\n→ STYLE DOMINATES: Writing tone is the major factor")
        elif demo_impact > style_impact * 1.5:
            print(f"\n→ DEMOGRAPHICS DOMINATE: Social bias is the major factor")
        else:
            print(f"\n→ BOTH MATTER: Style and demographics have similar impact")
    else:
        print("Insufficient data for comparison")